# import data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from tqdm import tqdm

In [ ]:
data = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
data.head(3)

In [ ]:
data.dtypes

all the features are numerical and there is not any missing value

In [ ]:
data.describe().T

see the Glucose, BloodPressure, SkinThickness, Insulin and BMI features, it shows min value as 0 !!! , this is crazy... 

no alive human have these level as 0, that means we need to treate those 0 values as missing values 

BMI = mass/hight^2, also can not be 0

In [ ]:
columns = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
data[columns] = data[columns].replace(0, np.NaN)
data.isnull().sum()

# So let's do class based Imputation 

In [ ]:
positive_data = data[data.Outcome == 1].copy()
negative_data = data[data.Outcome == 0].copy()

positive_data['Glucose'].fillna(positive_data['Glucose'].mean(), inplace = True)
positive_data['BloodPressure'].fillna(positive_data['BloodPressure'].mean(), inplace = True)
positive_data['SkinThickness'].fillna(positive_data['SkinThickness'].median(), inplace = True)
positive_data['Insulin'].fillna(positive_data['Insulin'].median(), inplace = True)
positive_data['BMI'].fillna(positive_data['BMI'].median(), inplace = True)

negative_data['Glucose'].fillna(negative_data['Glucose'].mean(), inplace = True)
negative_data['BloodPressure'].fillna(negative_data['BloodPressure'].mean(), inplace = True)
negative_data['SkinThickness'].fillna(negative_data['SkinThickness'].median(), inplace = True)
negative_data['Insulin'].fillna(negative_data['Insulin'].median(), inplace = True)
negative_data['BMI'].fillna(negative_data['BMI'].median(), inplace = True)

data = pd.concat([positive_data, negative_data])

# Knn With Column Standardization (~91.8% accuracy)

In [ ]:
# now except column "Outcome", apply StandardScaler
scaler = StandardScaler()

data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])

## Detect Outliers and remove them

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=20)
lof.fit(data)

inout_liers = lof.fit_predict(data)
print(inout_liers)

In [ ]:
inout_liers = pd.Series(inout_liers)
print(inout_liers.value_counts())

In [ ]:
inliers = inout_liers[inout_liers == 1]
outliers = inout_liers[inout_liers == -1]

data = data.loc[inliers.index]

In [ ]:
X = std_data.iloc[:, :-1]
Y = std_data.iloc[:, -1]

In [ ]:
metrics = ['canberra', 'braycurtis', 'chebyshev', 'correlation', 'dice', 'hamming', 'jaccard', 'kulsinski', 'minkowski', 'rogerstanimoto', 'russellrao', 'sokalmichener', 'sokalsneath', 'sqeuclidean']

ks = [v for v in range(5,80,2)]
ans = {"k":ks}
for m in metrics:
    ans[m] = []

for m in tqdm(metrics):
    for k in ks:
        knn = KNeighborsClassifier(k, metric=m);
        scores = cross_val_score(knn, X, Y, cv=4)
        ans[m].append(scores.mean())

In [ ]:
colors = np.random.rand(1, 4)

distances = list(ans.keys())
distances.remove("k")

for d in distances:
    plt.plot(np.arange(38), ans[d], label=d)

plt.legend(bbox_to_anchor=(1, 1), fancybox=True, shadow=True)
plt.show();

In [ ]:
max_metric = pd.DataFrame(ans).mean()[1:].idxmax()
print(max_metric)
pd.DataFrame(ans).mean()[max_metric]

In [ ]:
more_data = {"random_state":[s for s in range(100)], "k":[], "score":[]}
ks = [k for k in range(5,80,2)]

for r in tqdm(range(100)):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=r);
    scores = []
    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k, metric="canberra");
        knn.fit(X_train, Y_train);
        scores.append(knn.score(X_test, Y_test))
    more_data["k"].append(ks[scores.index(max(scores))])
    more_data["score"].append(max(scores))

In [ ]:
pd.DataFrame(more_data).sort_values(by="score", ascending=False).head()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=29);
knn = KNeighborsClassifier(n_neighbors=19, metric="canberra")
knn.fit(X_train, Y_train)
knn.score(X_test, Y_test)